In [19]:
import random
import numpy as np
import math
import copy

In [20]:
##### 读.fa
fa_in = open("D:/IntroToDataMining-CS412/final project/benchmark/dataset3/sequences.fa","r")
fa_Seq = []#定义一个列表，用于存储序列
fa_Num = 0
for line in fa_in.readlines():#每次从FASTA文件中读取一行
    line = line.rstrip()#去掉行末的换行符
    fa_Num = fa_Num + 1
    fa_Seq.append(line)
#fa_Seq
##### 读motif 长度
motiflen_in = open("D:/IntroToDataMining-CS412/final project/benchmark/dataset2/motiflength.txt","r")
motif_len = int(motiflen_in.readline())
#motif_len

In [21]:
def generate_prob(pro_matrix,k_mer): #k_mer 'CAAATCCC'
    '''
    给定pro_matrix的情况下，出现当前k_mer的概率
    '''
    dic = {'A':0,'T':1,'C':2,'G':3}
    p = 1
    for i in range(len(k_mer)):
        p = p * pro_matrix[dic[k_mer[i]]][i]
    return p

In [22]:
def background_prob(fa_Seq,sites,motif_len):
    dic = {'A':0,'T':1,'C':2,'G':3}
    background_c = np.zeros(4)
    for i in range(len(fa_Seq)):
        for j in range(len(fa_Seq[i])):
            if j not in list(range(sites[i],sites[i]+motif_len)):
                background_c[dic[fa_Seq[i][j]]] +=1
    background_p = background_c/sum(background_c)
    return background_p

### 无F

ToDo：converge（条件改为site不变多次）

In [ ]:
########################################## 无F
######### 设初值 
##### step1 随机选初始site
sites = [random.randint(0, (len(fa_Seq[0])-motif_len)) for i in range(fa_Num)] 
k_mers = []
for i in range(fa_Num):
    k_mers.append(list(fa_Seq[i][sites[i]:sites[i]+motif_len]))
f = [float('inf') for i in range(motif_len)]
A = []
B = []


######### 开始循环
hide_index = 0
equal_count = 0
while equal_count<3:  
    hide_same = True
    while hide_same:
        new_hide_index = random.randint(0,fa_Num-1)#随机隐藏一行
        if new_hide_index != hide_index:
            hide_same = False
            hide_index = new_hide_index
    temp_motif_matrix = k_mers[:hide_index]+k_mers[hide_index+1:] #生成临时motif matrix
    psudo_count_matrix = [[1 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
    
    for i in range(motif_len):
        for j in range(fa_Num-1):
            if temp_motif_matrix[j][i] == 'A':
                psudo_count_matrix[0][i] += 1
            elif temp_motif_matrix[j][i] == 'T':
                psudo_count_matrix[1][i] += 1
            elif temp_motif_matrix[j][i] == 'C':
                psudo_count_matrix[2][i] += 1
            elif temp_motif_matrix[j][i] == 'G':
                psudo_count_matrix[3][i] += 1
    psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix     
    pro_matrix = psudo_count_matrix/(fa_Num-1+4) #生成pro_matrix
    prob = [] #按概率选择hiden string的一个k_mer 用来更新对应行的 motif matrix
    hiden_string = fa_Seq[hide_index]
    
    for i in  range(len(fa_Seq[0])-motif_len+1):
        temp_k_mer = hiden_string[i:i+motif_len]
        prob.append(generate_prob(pro_matrix,temp_k_mer))
    # site 暂时更新
    updated_site = np.random.choice(len(fa_Seq[0])-motif_len+1,1,p = prob/sum(prob))[0]  
    temp_sites = copy.deepcopy(sites)
    temp_sites[hide_index] = updated_site # 更新site(temp_site)
    # k_mers 暂时更新
    temp_k_mers = copy.deepcopy(k_mers)
    temp_k_mers[hide_index] = list(fa_Seq[hide_index][sites[hide_index]:sites[hide_index]+motif_len]) # 新的k_mers(temp_k_mers)
    if temp_sites == sites:
        equal_count += 1
    else:
        equal_count = 0
    sites = copy.deepcopy(temp_sites)
    k_mers = copy.deepcopy(temp_k_mers)  
    A.append(sites)
    B.append(k_mers)


In [ ]:
B = B[-1]
M = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num):
        if B[j][i] == 'A':
            M[0][i] += 1
        elif B[j][i] == 'T':
            M[1][i] += 1
        elif B[j][i] == 'C':
            M[2][i] += 1
        elif B[j][i] == 'G':
            M[3][i] += 1
M = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
M/fa_Num

In [ ]:
(psudo_count_matrix/fa_Num).T

In [ ]:
A[-1]

### 用 每一位的entropy下降（F） 作site的更新标准

In [11]:
def F_given_model(temp_k_mers,background_p,f):
    '''
    计算F(计算F 若更新使F变大，更新；若不能变大，不更新)
    temp_k_mers : 
    [['A', 'C', 'G', 'T', 'T', 'A', 'C', 'T'],
     ['T', 'G', 'G', 'G', 'C', 'C', 'A', 'A'],
     ['T', 'G', 'A', 'C', 'A', 'T', 'G', 'A'],
     ['C', 'C', 'C', 'G', 'T', 'G', 'A', 'C'],
     ['G', 'A', 'T', 'C', 'C', 'A', 'C', 'A'],
     ['G', 'G', 'T', 'T', 'G', 'T', 'T', 'A'],
     ['T', 'G', 'G', 'T', 'G', 'T', 'C', 'A'],
     ['G', 'G', 'T', 'G', 'G', 'G', 'T', 'T'],
     ['A', 'T', 'T', 'C', 'C', 'G', 'C', 'A'],
     ['C', 'C', 'G', 'A', 'A', 'T', 'A', 'T']]
    background_p : [0.2496, 0.2492, 0.2516, 0.2496]
    '''
    dic = {'A':0,'T':1,'C':2,'G':3}
    sigma = ['A','T','C','G']
    
    #motif_count_matrix
    count_matrix = [[0 for i in range(motif_len)] for j in range(4)]
    for i in range(motif_len):
        for j in range(fa_Num):
            if temp_k_mers[j][i] == 'A':
                count_matrix[0][i] += 1
            elif temp_k_mers[j][i] == 'T':
                count_matrix[1][i] += 1
            elif temp_k_mers[j][i] == 'C':
                count_matrix[2][i] += 1
            elif temp_k_mers[j][i] == 'G':
                count_matrix[3][i] += 1
    count_matrix = np.array(count_matrix)
    '''
    [[3, 2, 2, 2, 3, 3, 4, 6],
     [3, 2, 5, 3, 3, 4, 3, 4],
     [3, 4, 2, 4, 4, 2, 4, 2],
     [4, 5, 4, 4, 3, 4, 2, 1]]
    '''
    
    #motif_probability_matrix
    pro_matrix = count_matrix/fa_Num
    '''
    [[0.23076923 0.15384615 0.15384615 0.15384615 0.23076923 0.23076923
      0.30769231 0.46153846]
     [0.23076923 0.15384615 0.38461538 0.23076923 0.23076923 0.30769231
      0.23076923 0.30769231]
     [0.23076923 0.30769231 0.15384615 0.30769231 0.30769231 0.15384615
      0.30769231 0.15384615]
     [0.30769231 0.38461538 0.30769231 0.30769231 0.23076923 0.30769231
      0.15384615 0.07692308]]
    ''' 
    
    flag = False #update
    temp_f = copy.deepcopy(f)
    for i in range(len(temp_k_mers[0])):
        newEntropy = 0
        for j in range(4):
            if pro_matrix[j][i] == 0:
                newEntropy -= 0
            else:
                newEntropy -= pro_matrix[j][i]*math.log(pro_matrix[j][i],2)
        temp_f[i] = newEntropy
        if newEntropy > f[i]:
            flag = True #don't update
    if flag:
        return flag,f 
    else:
        return flag,temp_f

In [17]:
########################################## 有F 且converge
######### 设初值 
##### step1 随机选初始site
sites = [random.randint(0, (len(fa_Seq[0])-motif_len)) for i in range(fa_Num)] 
k_mers = []
for i in range(fa_Num):
    k_mers.append(list(fa_Seq[i][sites[i]:sites[i]+motif_len]))
f = [float('inf') for i in range(motif_len)]
A = []
B = [[]]


######### 开始循环
equal_count = 0
while equal_count<3:      
    hide_index = random.randint(0,fa_Num-1)#随机隐藏一行
    temp_motif_matrix = k_mers[:hide_index]+k_mers[hide_index+1:] #生成临时motif matrix
    psudo_count_matrix = [[0.01 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
    for i in range(motif_len):
        for j in range(fa_Num-1):
            if temp_motif_matrix[j][i] == 'A':
                psudo_count_matrix[0][i] += 1
            elif temp_motif_matrix[j][i] == 'T':
                psudo_count_matrix[1][i] += 1
            elif temp_motif_matrix[j][i] == 'C':
                psudo_count_matrix[2][i] += 1
            elif temp_motif_matrix[j][i] == 'G':
                psudo_count_matrix[3][i] += 1
    psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix     
    pro_matrix = psudo_count_matrix/(fa_Num-1+4) #生成pro_matrix
    prob = [] #按概率选择hiden string的一个k_mer 用来更新对应行的 motif matrix
    hiden_string = fa_Seq[hide_index]
    
    for i in  range(len(fa_Seq[0])-motif_len+1):
        temp_k_mer = hiden_string[i:i+motif_len]
        prob.append(generate_prob(pro_matrix,temp_k_mer))
    # site 暂时更新
    updated_site = np.random.choice(len(fa_Seq[0])-motif_len+1,1,p = prob/sum(prob))[0]  
    temp_sites = copy.deepcopy(sites)
    temp_sites[hide_index] = updated_site # 更新site(temp_site)
    # k_mers 暂时更新
    temp_k_mers = copy.deepcopy(k_mers)
    temp_k_mers[hide_index] = list(fa_Seq[hide_index][sites[hide_index]:sites[hide_index]+motif_len]) # 新的k_mers(temp_k_mers)
    # 计算 background probability
    background_p = background_prob(fa_Seq,temp_sites,motif_len)

    # 计算entropy
    flag, f = F_given_model(temp_k_mers,background_p,f)
    #print(temp_F)
    if not flag:
        if temp_sites == sites:
            equal_count += 1
        else:
            equal_count = 0
        sites = copy.deepcopy(temp_sites)
        k_mers = copy.deepcopy(temp_k_mers)
        
    A.append(sites)
    B.append(k_mers)

996
997
998
999


In [ ]:
########################################## 有F 用iteration
######### 设初值 
##### step1 随机选初始site
sites = [random.randint(0, (len(fa_Seq[0])-motif_len)) for i in range(fa_Num)] 
k_mers = []
for i in range(fa_Num):
    k_mers.append(list(fa_Seq[i][sites[i]:sites[i]+motif_len]))
f = [float('inf') for i in range(motif_len)]
A = []
B = [[]]


######### 开始循环
iteration_n = 1000
for itera in range(iteration_n):    
    hide_index = random.randint(0,fa_Num-1)#随机隐藏一行
    temp_motif_matrix = k_mers[:hide_index]+k_mers[hide_index+1:] #生成临时motif matrix
    psudo_count_matrix = [[0.01 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
    for i in range(motif_len):
        for j in range(fa_Num-1):
            if temp_motif_matrix[j][i] == 'A':
                psudo_count_matrix[0][i] += 1
            elif temp_motif_matrix[j][i] == 'T':
                psudo_count_matrix[1][i] += 1
            elif temp_motif_matrix[j][i] == 'C':
                psudo_count_matrix[2][i] += 1
            elif temp_motif_matrix[j][i] == 'G':
                psudo_count_matrix[3][i] += 1
    psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix     
    pro_matrix = psudo_count_matrix/(fa_Num-1+4) #生成pro_matrix
    prob = [] #按概率选择hiden string的一个k_mer 用来更新对应行的 motif matrix
    hiden_string = fa_Seq[hide_index]
    
    for i in  range(len(fa_Seq[0])-motif_len+1):
        temp_k_mer = hiden_string[i:i+motif_len]
        prob.append(generate_prob(pro_matrix,temp_k_mer))
    # site 暂时更新
    updated_site = np.random.choice(len(fa_Seq[0])-motif_len+1,1,p = prob/sum(prob))[0]  
    temp_sites = copy.deepcopy(sites)
    temp_sites[hide_index] = updated_site # 更新site(temp_site)
    # k_mers 暂时更新
    temp_k_mers = copy.deepcopy(k_mers)
    temp_k_mers[hide_index] = list(fa_Seq[hide_index][sites[hide_index]:sites[hide_index]+motif_len]) # 新的k_mers(temp_k_mers)
    # 计算 background probability
    background_p = background_prob(fa_Seq,temp_sites,motif_len)

    # 计算entropy
    flag, f = F_given_model(temp_k_mers,background_p,f)
    #print(temp_F)
    if not flag:
        sites = copy.deepcopy(temp_sites)
        k_mers = copy.deepcopy(temp_k_mers)
        
    A.append(sites)
    #print(k_mers)
    if itera in [iteration_n-4,iteration_n-3,iteration_n-2,iteration_n-1]:
        print(itera)
        B.append(k_mers)

In [18]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix/fa_Num

array([[0.7 , 0.15, 0.45, 0.1 , 0.15, 0.2 , 0.2 , 0.35],
       [0.05, 0.35, 0.25, 0.4 , 0.2 , 0.25, 0.15, 0.15],
       [0.1 , 0.25, 0.25, 0.2 , 0.3 , 0.25, 0.4 , 0.25],
       [0.15, 0.25, 0.05, 0.3 , 0.35, 0.3 , 0.25, 0.25]])

In [19]:
A[-1]

[175,
 406,
 107,
 491,
 451,
 487,
 215,
 14,
 244,
 464,
 374,
 168,
 106,
 169,
 52,
 330,
 121,
 399,
 490,
 138]

### 用整个motif的entropy下降（F）作为site更新标准

In [28]:
def F_given_model(temp_k_mers,background_p,f):
    dic = {'A':0,'T':1,'C':2,'G':3}
    sigma = ['A','T','C','G']
    
    #motif_count_matrix
    count_matrix = [[0 for i in range(motif_len)] for j in range(4)]
    for i in range(motif_len):
        for j in range(fa_Num):
            if temp_k_mers[j][i] == 'A':
                count_matrix[0][i] += 1
            elif temp_k_mers[j][i] == 'T':
                count_matrix[1][i] += 1
            elif temp_k_mers[j][i] == 'C':
                count_matrix[2][i] += 1
            elif temp_k_mers[j][i] == 'G':
                count_matrix[3][i] += 1
    count_matrix = np.array(count_matrix)
    
    #motif_probability_matrix
    pro_matrix = count_matrix/fa_Num
    flag = False #update
    temp_f = copy.deepcopy(f)
    for i in range(len(temp_k_mers[0])):
        newEntropy = 0
        for j in range(4):
            if pro_matrix[j][i] == 0:
                newEntropy -= 0
            else:
                newEntropy -= pro_matrix[j][i]*math.log(pro_matrix[j][i],2)
    temp_f = newEntropy
    if newEntropy > f:
        flag = True #don't update
    if flag:
        return flag,f 
    else:
        return flag,temp_f

In [29]:
########################################## 有F 用iteration
######### 设初值 
##### step1 随机选初始site
sites = [random.randint(0, (len(fa_Seq[0])-motif_len)) for i in range(fa_Num)] 
k_mers = []
for i in range(fa_Num):
    k_mers.append(list(fa_Seq[i][sites[i]:sites[i]+motif_len]))
f = float('inf')
A = []
B = [[]]


######### 开始循环
iteration_n = 1000
for itera in range(iteration_n):    
    hide_index = random.randint(0,fa_Num-1)#随机隐藏一行
    temp_motif_matrix = k_mers[:hide_index]+k_mers[hide_index+1:] #生成临时motif matrix
    psudo_count_matrix = [[0.01 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
    for i in range(motif_len):
        for j in range(fa_Num-1):
            if temp_motif_matrix[j][i] == 'A':
                psudo_count_matrix[0][i] += 1
            elif temp_motif_matrix[j][i] == 'T':
                psudo_count_matrix[1][i] += 1
            elif temp_motif_matrix[j][i] == 'C':
                psudo_count_matrix[2][i] += 1
            elif temp_motif_matrix[j][i] == 'G':
                psudo_count_matrix[3][i] += 1
    psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix     
    pro_matrix = psudo_count_matrix/(fa_Num-1+4) #生成pro_matrix
    prob = [] #按概率选择hiden string的一个k_mer 用来更新对应行的 motif matrix
    hiden_string = fa_Seq[hide_index]
    
    for i in  range(len(fa_Seq[0])-motif_len+1):
        temp_k_mer = hiden_string[i:i+motif_len]
        prob.append(generate_prob(pro_matrix,temp_k_mer))
    # site 暂时更新
    updated_site = np.random.choice(len(fa_Seq[0])-motif_len+1,1,p = prob/sum(prob))[0]  
    temp_sites = copy.deepcopy(sites)
    temp_sites[hide_index] = updated_site # 更新site(temp_site)
    # k_mers 暂时更新
    temp_k_mers = copy.deepcopy(k_mers)
    temp_k_mers[hide_index] = list(fa_Seq[hide_index][sites[hide_index]:sites[hide_index]+motif_len]) # 新的k_mers(temp_k_mers)
    # 计算 background probability
    background_p = background_prob(fa_Seq,temp_sites,motif_len)

    # 计算entropy
    flag, f = F_given_model(temp_k_mers,background_p,f)
    #print(temp_F)
    if not flag:
        sites = copy.deepcopy(temp_sites)
        k_mers = copy.deepcopy(temp_k_mers)
        
    A.append(sites)
    #print(k_mers)
    if itera in [iteration_n-4,iteration_n-3,iteration_n-2,iteration_n-1]:
        print(itera)
        B.append(k_mers)

996
997
998
999


In [27]:
########################################## 有F 且converge
######### 设初值 
##### step1 随机选初始site
sites = [random.randint(0, (len(fa_Seq[0])-motif_len)) for i in range(fa_Num)] 
k_mers = []
for i in range(fa_Num):
    k_mers.append(list(fa_Seq[i][sites[i]:sites[i]+motif_len]))
f = float('inf')
A = []
B = [[]]


######### 开始循环
equal_count = 0
while equal_count<3:      
    hide_index = random.randint(0,fa_Num-1)#随机隐藏一行
    temp_motif_matrix = k_mers[:hide_index]+k_mers[hide_index+1:] #生成临时motif matrix
    psudo_count_matrix = [[0.01 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
    for i in range(motif_len):
        for j in range(fa_Num-1):
            if temp_motif_matrix[j][i] == 'A':
                psudo_count_matrix[0][i] += 1
            elif temp_motif_matrix[j][i] == 'T':
                psudo_count_matrix[1][i] += 1
            elif temp_motif_matrix[j][i] == 'C':
                psudo_count_matrix[2][i] += 1
            elif temp_motif_matrix[j][i] == 'G':
                psudo_count_matrix[3][i] += 1
    psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix     
    pro_matrix = psudo_count_matrix/(fa_Num-1+4) #生成pro_matrix
    prob = [] #按概率选择hiden string的一个k_mer 用来更新对应行的 motif matrix
    hiden_string = fa_Seq[hide_index]
    
    for i in  range(len(fa_Seq[0])-motif_len+1):
        temp_k_mer = hiden_string[i:i+motif_len]
        prob.append(generate_prob(pro_matrix,temp_k_mer))
    # site 暂时更新
    updated_site = np.random.choice(len(fa_Seq[0])-motif_len+1,1,p = prob/sum(prob))[0]  
    temp_sites = copy.deepcopy(sites)
    temp_sites[hide_index] = updated_site # 更新site(temp_site)
    # k_mers 暂时更新
    temp_k_mers = copy.deepcopy(k_mers)
    temp_k_mers[hide_index] = list(fa_Seq[hide_index][sites[hide_index]:sites[hide_index]+motif_len]) # 新的k_mers(temp_k_mers)
    # 计算 background probability
    background_p = background_prob(fa_Seq,temp_sites,motif_len)

    # 计算entropy
    flag, f = F_given_model(temp_k_mers,background_p,f)
    #print(temp_F)
    if not flag:
        if temp_sites == sites:
            equal_count += 1
        else:
            equal_count = 0
        sites = copy.deepcopy(temp_sites)
        k_mers = copy.deepcopy(temp_k_mers)
        
    A.append(sites)
    B.append(k_mers)

KeyboardInterrupt: 

In [30]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix/fa_Num

array([[0.05, 0.35, 0.3 , 0.5 , 0.3 , 0.5 , 0.35, 0.  ],
       [0.4 , 0.35, 0.3 , 0.05, 0.2 , 0.2 , 0.3 , 0.  ],
       [0.25, 0.2 , 0.15, 0.05, 0.3 , 0.3 , 0.  , 0.  ],
       [0.3 , 0.1 , 0.25, 0.4 , 0.2 , 0.  , 0.35, 1.  ]])

In [31]:
A[-1]

[58,
 15,
 132,
 211,
 417,
 350,
 333,
 412,
 215,
 43,
 406,
 336,
 50,
 478,
 311,
 144,
 231,
 375,
 68,
 430]

### 用之前的 log-odds上升/大于0（F） 作为更新标准

In [37]:
def F_given_model(temp_k_mers,background_p):
    '''
    计算F(计算F 若更新使F变大，更新；若不能变大，不更新)
    '''
    dic = {'A':0,'T':1,'C':2,'G':3}
    sigma = ['A','T','C','G']
    
    #motif_count_matrix
    count_matrix = [[0 for i in range(motif_len)] for j in range(4)]
    for i in range(motif_len):
        for j in range(fa_Num):
            if temp_k_mers[j][i] == 'A':
                count_matrix[0][i] += 1
            elif temp_k_mers[j][i] == 'T':
                count_matrix[1][i] += 1
            elif temp_k_mers[j][i] == 'C':
                count_matrix[2][i] += 1
            elif temp_k_mers[j][i] == 'G':
                count_matrix[3][i] += 1
    count_matrix = np.array(count_matrix)
    
    #motif_probability_matrix
    pro_matrix = count_matrix/fa_Num
    '''
    [[0.23076923 0.15384615 0.15384615 0.15384615 0.23076923 0.23076923
      0.30769231 0.46153846]
     [0.23076923 0.15384615 0.38461538 0.23076923 0.23076923 0.30769231
      0.23076923 0.30769231]
     [0.23076923 0.30769231 0.15384615 0.30769231 0.30769231 0.15384615
      0.30769231 0.15384615]
     [0.30769231 0.38461538 0.30769231 0.30769231 0.23076923 0.30769231
      0.15384615 0.07692308]]
    '''
    #print(count_matrix)
    F = 0
    for i in range(len(temp_k_mers[0])):
        for j in range(4):
            if count_matrix[j][i] == 0:
                F += 0
            else:
                F += count_matrix[j][i]*math.log2(pro_matrix[j][i]/background_p[j])
    return F

In [38]:
########################################## 有F - iteration
######### 设初值 
##### step1 随机选初始site
sites = [random.randint(0, (len(fa_Seq[0])-motif_len)) for i in range(fa_Num)] 
k_mers = []
for i in range(fa_Num):
    k_mers.append(list(fa_Seq[i][sites[i]:sites[i]+motif_len]))
f = -100
A = []
B = [[]]


######### 开始循环
iteration_n = 100000
for itera in range(iteration_n):    
    hide_index = random.randint(0,fa_Num-1)#随机隐藏一行
    temp_motif_matrix = k_mers[:hide_index]+k_mers[hide_index+1:] #生成临时motif matrix
    psudo_count_matrix = [[0.01 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
    for i in range(motif_len):
        for j in range(fa_Num-1):
            if temp_motif_matrix[j][i] == 'A':
                psudo_count_matrix[0][i] += 1
            elif temp_motif_matrix[j][i] == 'T':
                psudo_count_matrix[1][i] += 1
            elif temp_motif_matrix[j][i] == 'C':
                psudo_count_matrix[2][i] += 1
            elif temp_motif_matrix[j][i] == 'G':
                psudo_count_matrix[3][i] += 1
    psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix     
    pro_matrix = psudo_count_matrix/(fa_Num-1+4) #生成pro_matrix
    prob = [] #按概率选择hiden string的一个k_mer 用来更新对应行的 motif matrix
    hiden_string = fa_Seq[hide_index]
    
    for i in  range(len(fa_Seq[0])-motif_len+1):
        temp_k_mer = hiden_string[i:i+motif_len]
        prob.append(generate_prob(pro_matrix,temp_k_mer))
    # site 暂时更新
    updated_site = np.random.choice(len(fa_Seq[0])-motif_len+1,1,p = prob/sum(prob))[0]  
    temp_sites = copy.deepcopy(sites)
    temp_sites[hide_index] = updated_site # 更新site(temp_site)
    # k_mers 暂时更新
    temp_k_mers = copy.deepcopy(k_mers)
    temp_k_mers[hide_index] = list(fa_Seq[hide_index][sites[hide_index]:sites[hide_index]+motif_len]) # 新的k_mers(temp_k_mers)

    # 计算 background probability
    dic = {'A':0,'T':1,'C':2,'G':3}
    background_c = [0 for i in range(4)]
    background_c = np.array(background_c)
    for i in fa_Seq:
        for j in i:
            background_c[dic[j]] +=1
    background_p = background_c/sum(background_c)

    temp_F = F_given_model(temp_k_mers,background_p)
    #print(temp_F)
    if temp_F > f:
        sites = copy.deepcopy(temp_sites)
        k_mers = copy.deepcopy(temp_k_mers)
        f = temp_F
   
    A.append(sites)
    #print(k_mers)
    if itera in [iteration_n-4,iteration_n-3,iteration_n-2,iteration_n-1]:
        print(itera)
        B.append(k_mers)

99996
99997
99998
99999


In [39]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix/fa_Num

array([[0.25, 0.05, 0.15, 0.3 , 0.4 , 0.25, 0.25, 0.15],
       [0.25, 0.35, 0.2 , 0.15, 0.25, 0.2 , 0.2 , 0.45],
       [0.3 , 0.4 , 0.25, 0.1 , 0.2 , 0.25, 0.1 , 0.3 ],
       [0.2 , 0.2 , 0.4 , 0.45, 0.15, 0.3 , 0.45, 0.1 ]])

In [40]:
A[-1]

[110,
 12,
 127,
 103,
 426,
 69,
 34,
 434,
 372,
 116,
 296,
 476,
 401,
 373,
 91,
 321,
 302,
 248,
 42,
 101]

In [183]:
def F_given_model(temp_k_mers,background_p):
    '''
    计算F(计算F 若更新使F变大，更新；若不能变大，不更新)
    temp_k_mers : 
    [['A', 'C', 'G', 'T', 'T', 'A', 'C', 'T'],
     ['T', 'G', 'G', 'G', 'C', 'C', 'A', 'A'],
     ['T', 'G', 'A', 'C', 'A', 'T', 'G', 'A'],
     ['C', 'C', 'C', 'G', 'T', 'G', 'A', 'C'],
     ['G', 'A', 'T', 'C', 'C', 'A', 'C', 'A'],
     ['G', 'G', 'T', 'T', 'G', 'T', 'T', 'A'],
     ['T', 'G', 'G', 'T', 'G', 'T', 'C', 'A'],
     ['G', 'G', 'T', 'G', 'G', 'G', 'T', 'T'],
     ['A', 'T', 'T', 'C', 'C', 'G', 'C', 'A'],
     ['C', 'C', 'G', 'A', 'A', 'T', 'A', 'T']]
    background_p : [0.2496, 0.2492, 0.2516, 0.2496]
    '''
    dic = {'A':0,'T':1,'C':2,'G':3}
    sigma = ['A','T','C','G']
    
    #motif_count_matrix
    count_matrix = [[0 for i in range(motif_len)] for j in range(4)]
    for i in range(motif_len):
        for j in range(fa_Num):
            if temp_k_mers[j][i] == 'A':
                count_matrix[0][i] += 1
            elif temp_k_mers[j][i] == 'T':
                count_matrix[1][i] += 1
            elif temp_k_mers[j][i] == 'C':
                count_matrix[2][i] += 1
            elif temp_k_mers[j][i] == 'G':
                count_matrix[3][i] += 1
    count_matrix = np.array(count_matrix)
    '''
    [[3, 2, 2, 2, 3, 3, 4, 6],
     [3, 2, 5, 3, 3, 4, 3, 4],
     [3, 4, 2, 4, 4, 2, 4, 2],
     [4, 5, 4, 4, 3, 4, 2, 1]]
    '''
    
    #motif_probability_matrix
    pro_matrix = count_matrix/fa_Num
    '''
    [[0.23076923 0.15384615 0.15384615 0.15384615 0.23076923 0.23076923
      0.30769231 0.46153846]
     [0.23076923 0.15384615 0.38461538 0.23076923 0.23076923 0.30769231
      0.23076923 0.30769231]
     [0.23076923 0.30769231 0.15384615 0.30769231 0.30769231 0.15384615
      0.30769231 0.15384615]
     [0.30769231 0.38461538 0.30769231 0.30769231 0.23076923 0.30769231
      0.15384615 0.07692308]]
    '''
    #print(count_matrix)
    F = 0
    for i in range(len(temp_k_mers[0])):
        for j in range(4):
            if count_matrix[j][i] == 0:
                F += 0
            else:
                F += count_matrix[j][i]*math.log2(pro_matrix[j][i]/background_p[j])
    return F

In [47]:
B = B[-1]
psudo_count_matrix = [[0 for i in range(motif_len)] for j in range(4)] #生成psudo_count_matrix
for i in range(motif_len):
    for j in range(fa_Num):
        if B[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif B[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif B[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif B[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix   
psudo_count_matrix/fa_Num

#341444

array([[0.25, 0.2 , 0.05, 0.25, 0.35, 0.1 , 0.55, 0.2 ],
       [0.35, 0.35, 0.4 , 0.3 , 0.4 , 0.25, 0.05, 0.35],
       [0.35, 0.35, 0.25, 0.2 , 0.05, 0.45, 0.3 , 0.05],
       [0.05, 0.1 , 0.3 , 0.25, 0.2 , 0.2 , 0.1 , 0.4 ]])